### 3.4.1. 分类问题
独热编码（one-hot encoding）。 独热编码是一个向量，它的分量和类别一样多。 类别对应的分量设置为1，其他所有分量设置为0。

### 3.4.4. softmax运算
要将输出视为概率，我们必须保证在任何数据上的输出都是非负的且总和为1。 此外，我们需要一个训练的目标函数，来激励模型精准地估计概率。 例如， 在分类器输出0.5的所有样本中，我们希望这些样本是刚好有一半实际上属于预测的类别。 这个属性叫做校准（calibration）。  
社会科学家邓肯·卢斯于1959年在选择模型（choice model）的理论基础上 发明的softmax函数正是这样做的： softmax函数能够将未规范化的预测变换为非负数并且总和为1，同时让模型保持 可导的性质。 为了完成这一目标，我们首先对每个未规范化的预测求幂，这样可以确保输出非负。 为了确保最终输出的概率值总和为1，我们再让每个求幂后的结果除以它们的总和。如下式：
$$\hat{\mathbf{y}} = \mathrm{softmax}(\mathbf{o})\quad \text{其中}\quad \hat{y}_j = \frac{\exp(o_j)}{\sum_k \exp(o_k)}$$
只会确定分配给每个类别的概率。 因此，在预测过程中，我们仍然可以用下式来选择最有可能的类别。  
$$\operatorname*{argmax}_j \hat y_j = \operatorname*{argmax}_j o_j.$$
尽管softmax是一个非线性函数，但softmax回归的输出仍然由输入特征的仿射变换决定。 因此，softmax回归是一个线性模型（linear model）。

### 3.4.6. 损失函数
cross-entropy loss:$$l(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{j=1}^q y_j \log \hat{y}_j.$$
softmax和cross-cntropy loss结合及其求导:
$$\begin{split}\begin{aligned}
l(\mathbf{y}, \hat{\mathbf{y}}) &=  - \sum_{j=1}^q y_j \log \frac{\exp(o_j)}{\sum_{k=1}^q \exp(o_k)} \\
&= \sum_{j=1}^q y_j \log \sum_{k=1}^q \exp(o_k) - \sum_{j=1}^q y_j o_j\\
&= \log \sum_{k=1}^q \exp(o_k) - \sum_{j=1}^q y_j o_j.
\end{aligned}\end{split}$$
考虑相对于任何未规范化的预测$o_{j}$的导数，我们得到：$$\partial_{o_j} l(\mathbf{y}, \hat{\mathbf{y}}) = \frac{\exp(o_j)}{\sum_{k=1}^q \exp(o_k)} - y_j = \mathrm{softmax}(\mathbf{o})_j - y_j.$$
换句话说，导数是我们softmax模型分配的概率与实际发生的情况（由独热标签向量表示）之间的差异。 从这个意义上讲，这与我们在回归中看到的非常相似， 其中梯度是观测值$y$和估计值$\hat{y}$之间的差异。 这不是巧合，在任何指数族分布模型中 （参见本书附录中关于数学分布的一节）， 对数似然的梯度正是由此得出的。 这使梯度计算在实践中变得容易很多。

### 3.4.7. 信息论基础
熵：$$H[P] = \sum_j - P(j) \log P(j).$$  其中P为分布函数  
信息量：$$\log \frac{1}{P(j)} = -\log P(j)$$
如果把熵$H(P)$想象为“知道真实概率的人所经历的惊异程度”，那么什么是交叉熵？ 交叉熵从$P$
到$Q$，记为$H(P,Q)$。 我们可以把交叉熵想象为“主观概率为$Q$的观察者在看到根据概率$P$生成的数据时的预期惊异”。 当$P=Q$时，交叉熵达到最低。 在这种情况下，从$P$到$Q$的交叉熵是$H(P,P) = H(P)$。

简而言之，我们可以从两方面来考虑交叉熵分类目标： （i）最大化观测数据的似然；（ii）最小化传达标签所需的惊异。

### Practice

#### 1. 我们可以更深入地探讨指数族与softmax之间的联系。
1. **计算softmax交叉熵损失 $l(\mathbf{y}, \hat{\mathbf{y}})$ 的二阶导数。**

   **解答：**
   设 $\hat{\mathbf{y}} = \text{softmax}(\mathbf{z})$，目标类别为 $\mathbf{y}$，则交叉熵损失为：
   $$
   l(\mathbf{y}, \hat{\mathbf{y}}) = -\sum_{i} y_i \log(\hat{y}_i)
   $$
   其中 $\hat{y}_i = \frac{e^{z_i}}{\sum_{j} e^{z_j}}$。

   二阶导数的计算步骤：
   - 一阶导数：
     $$
     \frac{\partial l}{\partial z_k} = \hat{y}_k - y_k
     $$
   - 二阶导数：
     对每个 $z_i$ 和 $z_j$，二阶导数为：
     $$
     \frac{\partial^2 l}{\partial z_i \partial z_j} =
     \begin{cases} 
     \hat{y}_i (1 - \hat{y}_i), & \text{if } i = j \\
     -\hat{y}_i \hat{y}_j, & \text{if } i \neq j
     \end{cases}
     $$
   结果是一个对角线主元素为 $\hat{y}_i (1 - \hat{y}_i)$，非对角线元素为 $-\hat{y}_i \hat{y}_j$ 的对称矩阵。

2. **计算softmax($\mathbf{o}$)给出的分布方差，并与上述计算的二阶导数匹配。**

   **解答：**
   softmax的输出 $\hat{y}_i = \frac{e^{o_i}}{\sum_{j} e^{o_j}}$ 是一个概率分布。其方差为：
   $$
   \text{Var}(\hat{y}_i) = \hat{y}_i (1 - \hat{y}_i)
   $$
   二阶导数矩阵的对角线部分（$\frac{\partial^2 l}{\partial z_i^2}$）正好等于 $\text{Var}(\hat{y}_i)$，非对角部分（$\frac{\partial^2 l}{\partial z_i \partial z_j}$）反映了分布中的协方差。

---

#### 2. 假设我们有三个类发生的概率相等，即概率向量是 $\left(\frac{1}{3}, \frac{1}{3}, \frac{1}{3}\right)$。
1. **如果我们尝试为它设计二进制代码，有什么问题？**

   **解答：**
   二进制编码的长度必须是整数，而均匀分布下的三个类的信息熵为：
   $$
   H = -\sum_{i=1}^3 \frac{1}{3} \log_2 \frac{1}{3} = \log_2 3 \approx 1.585
   $$
   信息熵不是整数，因此无法直接为其设计一个固定长度的二进制编码。此外，若使用固定长度编码（例如2比特），将导致编码效率下降。

2. **请设计一个更好的代码。提示：如果我们尝试编码两个独立的观察结果会发生什么？如果我们联合编码 $n$ 个观察值怎么办？**

   **解答：**
   - **联合编码两个观察结果：**
     观察两次的联合概率为 $\left(\frac{1}{3}, \frac{1}{3}, \frac{1}{3}\right) \times \left(\frac{1}{3}, \frac{1}{3}, \frac{1}{3}\right)$，对应的熵为：
     $$
     H = 2 \log_2 3 \approx 3.17
     $$
     此时，可以使用4比特编码联合观察结果，利用小数比特部分实现更高的编码效率。
   - **联合编码 $n$ 个观察值：**
     对于 $n$ 次联合观察，总熵为 $n \log_2 3$。随着 $n$ 增大，熵更接近整数，可以通过霍夫曼编码等方法实现接近最优的编码效率。

---

#### 3. softmax是对上面介绍的映射的误称（虽然深度学习领域中很多人都使用这个名字）。真正的softmax被定义为：
$$
\text{RealSoftMax}(a, b) = \log\left(\exp(a) + \exp(b)\right)
$$
1. **证明 $\text{RealSoftMax}(a, b) > \max(a, b)$。**

   **解答：**
   根据定义：
   $$
   \text{RealSoftMax}(a, b) = \log\left(\exp(a) + \exp(b)\right)
   $$
   设 $\max(a, b) = a$（不失一般性），则：
   $$
   \exp(a) + \exp(b) > \exp(a)
   $$
   取对数，两边均大于 $\log(\exp(a)) = a$，因此：
   $$
   \text{RealSoftMax}(a, b) > \max(a, b)
   $$

2. **证明 $\lambda^{-1} \text{RealSoftMax}(\lambda a, \lambda b) > \max(a, b)$ 成立，前提是 $\lambda > 0$。**

   **解答：**
   根据定义：
   $$
   \lambda^{-1} \text{RealSoftMax}(\lambda a, \lambda b) = \lambda^{-1} \log\left(\exp(\lambda a) + \exp(\lambda b)\right)
   $$
   设 $\max(a, b) = a$，则 $\exp(\lambda a) + \exp(\lambda b) > \exp(\lambda a)$，且：
   $$
   \lambda^{-1} \log\left(\exp(\lambda a)\right) = a
   $$
   因此：
   $$
   \lambda^{-1} \text{RealSoftMax}(\lambda a, \lambda b) > a = \max(a, b)
   $$

3. **证明对于 $\lambda \to \infty$，有 $\lambda^{-1} \text{RealSoftMax}(\lambda a, \lambda b) \to \max(a, b)$。**

   **解答：**
   当 $\lambda \to \infty$ 时，$\exp(\lambda a)$ 和 $\exp(\lambda b)$ 中较大的指数项将主导 $\exp(\lambda a) + \exp(\lambda b)$，因此：
   $$
   \text{RealSoftMax}(\lambda a, \lambda b) \approx \lambda \max(a, b)
   $$
   两边同时除以 $\lambda$，得：
   $$
   \lambda^{-1} \text{RealSoftMax}(\lambda a, \lambda b) \to \max(a, b)
   $$

4. **soft-min会是什么样子？**

   **解答：**
   soft-min的定义类似于softmax，但对较小的值进行映射：
   $$
   \text{SoftMin}(a, b) = -\log\left(\exp(-a) + \exp(-b)\right)
   $$

5. **将其扩展到两个以上的数字。**

   **解答：**
   对于多个数字 $\{x_1, x_2, \dots, x_n\}$，RealSoftMax扩展为：
   $$
   \text{RealSoftMax}(x_1, x_2, \dots, x_n) = \log\left(\sum_{i=1}^n \exp(x_i)\right)
   $$
   对应的soft-min扩展为：
   $$
   \text{SoftMin}(x_1, x_2, \dots, x_n) = -\log\left(\sum_{i=1}^n \exp(-x_i)\right)
   $$